# High-level RNN TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params_lstm import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
Tensorflow:  1.4.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
def create_symbol(CUDNN=True, 
                  maxf=MAXFEATURES, edim=EMBEDSIZE, nhid=NUMHIDDEN, batchs=BATCHSIZE):
    word_vectors = tf.contrib.layers.embed_sequence(X, vocab_size=maxf, embed_dim=edim)
    word_list = tf.unstack(word_vectors, axis=1)
    
    if not CUDNN:
        cell = tf.contrib.rnn.GRUCell(nhid)
        outputs, states = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)
    else:
        # Using cuDNN since vanilla RNN
        cudnn_cell = tf.contrib.cudnn_rnn.CudnnGRU(num_layers=1, 
                                                   num_units=nhid, 
                                                   input_size=edim)
        params_size_t = cudnn_cell.params_size()
        params = tf.Variable(tf.random_uniform([params_size_t], -0.1, 0.1), validate_shape=False)   
        input_h = tf.Variable(tf.zeros([1, batchs, nhid]))
        outputs, states = cudnn_cell(input_data=word_list,
                                     input_h=input_h,
                                     params=params)
        logits = tf.layers.dense(outputs[-1], 2, activation=None, name='output')
    return logits

In [5]:
def init_model(m, y, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(lr, b1, b2, eps)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 5.9 s, sys: 417 ms, total: 6.32 s
Wall time: 6.32 s


In [7]:
%%time
# Place-holders
X = tf.placeholder(tf.int32, shape=[None, MAXLEN])
y = tf.placeholder(tf.int32, shape=[None])
sym = create_symbol()

CPU times: user 737 ms, sys: 76.1 ms, total: 814 ms
Wall time: 820 ms


In [8]:
%%time
model = init_model(sym, y)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

CPU times: user 836 ms, sys: 693 ms, total: 1.53 s
Wall time: 1.54 s


In [9]:
%%time
# Main training loop: 22s
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.84375
1 Train accuracy: 0.96875
2 Train accuracy: 0.984375
CPU times: user 19 s, sys: 2.77 s, total: 21.8 s
Wall time: 22.2 s


In [10]:
%%time
# Main evaluation loop: 9.19s
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym, 1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 8.67 s, sys: 651 ms, total: 9.32 s
Wall time: 9.19 s


In [12]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.8598557692307692
